In [3]:
#Install The required libraries and import them.

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import pyspark.sql.functions as F 
from sqlalchemy import create_engine
import psycopg2
import logging

In [2]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Ingestion of CSV Files") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

In [ ]:
#Create a validation function based on rules. For this example, the function will validate if the columns of a table are empty on certain rows.
def nullCount(df, inputColumns):
    df_valid = df #Initially equal to the original dataset.
    df_invalid = df.filter("1=0").withColumn("validationMessage",F.lit("")) #Creating empty dataframe with the dataset schema and validation message
    columns = inputColumns
    source_count=df_valid.count()

    if columns:
        #Create the valid and not valid conditions
        query_null = " IS NULL OR ".join(columns) + " IS NULL"
        query_not_null = " IS NOT NULL AND ".join(columns) + " IS NOT NULL"
        #Filter the dataframe from valid and not valid conditions 
        df_nullCount_invalid = df_valid.filter(query_null)
        df_nullCount_valid = df_valid.filter(query_not_null)
    
        #Split the table on valid and not valid tables.
        df_valid = df_nullCount_valid
        df_invalid = df_invalid.union(df_nullCount_invalid.withColumn("validationMessage",F.lit("this record was removed by NullCount validation")))
   
    return df_valid, df_invalid

In [ ]:
#Write Tables on a PostgreSQL Database
def write_to_PostgreSQL (df,table_name,engine):
    pandas_df = df.toPandas()
    pandas_df.to_sql(table_name, engine, if_exists='overwrite', index=False)
